In [19]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import operator
from statistics import mean,pstdev
import numpy as np
import random
import copy
from math import sqrt
from numpy import linalg as LA

In [3]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [4]:
# cd ..

In [5]:
#drive.unmount()

In [6]:
#cd drive/My Drive/RecommenderSystems And TopicModelling Lab2

In [9]:
path="data/ml-latest-small/"
path_preloaded="data/"
ratings_path=path+"ratings.csv"
movies_path=path+"movies.csv"

In [10]:
ratings_df=pd.read_csv(ratings_path)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
movie_indices=[]
inverse_movie_map={}
movies_df=pd.read_csv(movies_path)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
#np.save(path_preloaded+'user_sim_matrix_cosine',user_sim_matrix_cosine)
#np.save(path_preloaded+'user_sim_matrix_pearson',user_sim_matrix_pearson)
ratings_matrix_train=np.load(path_preloaded+'ratings_matrix_train.npy')
train_nos=np.load(path_preloaded+'train_nos.npy')
test_nos=np.load(path_preloaded+'test_nos.npy')


In [13]:
correct_predictions=[]
for i in test_nos:
    correct_predictions.append(ratings_df['rating'][i])

In [14]:
def mean_with_nan(list_a):
    count=0
    sum=0.0
    for i in list_a:
        if not np.isnan(i):
            #print(i)
            count+=1
            sum+=i
    #print(sum) 
    if(count==0):
        return 0
    return sum/count     

In [15]:
for i in range(len(movies_df)):
    movie_indices.append(movies_df['movieId'][i])
    inverse_movie_map[movies_df['movieId'][i]]=i
number_of_items=len(movie_indices)

In [16]:
number_of_users={}
for i in range(len(ratings_df)):
    number_of_users[ratings_df['userId'][i]]=1
number_of_users=len(number_of_users)

In [17]:
obs_entries=[]
for i in range(number_of_users):
    for j in range(number_of_items):
        if not(np.isnan(ratings_matrix_train[i][j])):
            obs_entries.append((i,j))

In [18]:
np.save(path+'obs_entries',obs_entries)

In [15]:
ratings_matrix_filled=copy.deepcopy(ratings_matrix_train)

In [16]:
for i in range (number_of_users):
  mean_user=mean_with_nan(ratings_matrix_train[i])
  for j in range (number_of_items):
    if(np.isnan(ratings_matrix_filled[i][j])):
      ratings_matrix_filled[i][j]=mean_user

In [16]:
u,w,v=LA.svd(ratings_matrix_filled)

In [26]:
w

array([9.00899042e+03, 5.09910303e+01, 3.68548596e+01, 3.46277166e+01,
       3.23077166e+01, 3.11627111e+01, 3.02597519e+01, 2.92557407e+01,
       2.80957246e+01, 2.76934011e+01, 2.72936441e+01, 2.70381836e+01,
       2.63816556e+01, 2.55977758e+01, 2.55690623e+01, 2.48103934e+01,
       2.44354871e+01, 2.41866135e+01, 2.37509526e+01, 2.35433677e+01,
       2.31455902e+01, 2.28093020e+01, 2.24619118e+01, 2.23838926e+01,
       2.21618795e+01, 2.18469602e+01, 2.16231278e+01, 2.14186470e+01,
       2.10878582e+01, 2.06940180e+01, 2.05850659e+01, 2.02712025e+01,
       2.01967898e+01, 2.00980616e+01, 1.99804288e+01, 1.97648277e+01,
       1.95990246e+01, 1.94422200e+01, 1.91529330e+01, 1.89366547e+01,
       1.88675299e+01, 1.88298839e+01, 1.85656888e+01, 1.82560512e+01,
       1.81343361e+01, 1.79157175e+01, 1.78910949e+01, 1.77178196e+01,
       1.76171474e+01, 1.75940089e+01, 1.74154826e+01, 1.73281239e+01,
       1.71892734e+01, 1.71218184e+01, 1.69617044e+01, 1.68580492e+01,
      

In [0]:
#count_w=0

In [17]:
count_w=0
for i in w:
  if i>5:
    count_w+=1
print(count_w)    

346


In [18]:
k=count_w

In [25]:
k=346

In [26]:
def rmse_error(a,b):
    error_sum=0.0
    for i in range(len(a)):
        error_sum+=(a[i]-b[i])*(a[i]-b[i])
    error_sum/=len(a)
    return np.sqrt(error_sum)       

In [30]:
def GradientDescent(R,k,lr,epochs):
    number_of_items=len(R[0])
    number_of_users=len(R[:,0])
    U=np.random.rand(number_of_users,k)
    V=np.random.rand(number_of_items,k)
    error_mat=np.zeros(shape=(number_of_users,number_of_items))
    num_iters=0
    while(num_iters<epochs):
        num_iters+=1
        
        predicted_matrix=np.dot(U,np.transpose(V))


        predicted_matrix=np.dot(U,np.transpose(V))
        error_mat=R-predicted_matrix
        error=0
        for tups in obs_entries:
            x=tups[0]
            y=tups[1]
            error+=(R[x][y]-predicted_matrix[x][y])*(R[x][y]-predicted_matrix[x][y])
        error=sqrt(error) 
        if(num_iters%100==0):
            error=0
            for tups in obs_entries:
                x=tups[0]
                y=tups[1]
                error+=(R[x][y]-predicted_matrix[x][y])*(R[x][y]-predicted_matrix[x][y])
            error=sqrt(error)            
            result_gd_batch=[]
            for i in test_nos:


                movie_no=inverse_movie_map[ratings_df['movieId'][i]]
                user_no=ratings_df['userId'][i]-1
                result_gd_batch.append(predicted_matrix[user_no][movie_no])
            print("Number of iterations: "+str(num_iters)+" Frob Error: "+str(error)+" RMS Error: "+str(rmse_error(correct_predictions,result_gd_batch)))
        #      print("Error is " +str(error) )
        U_add=np.zeros(shape=(number_of_users,k))
        V_add=np.zeros(shape=(number_of_items,k))
        for tups in obs_entries:
            x=tups[0]
            y=tups[1]
            for q_ind in range(k):
                U_add[x][q_ind]+=error_mat[x][y]*V[y][q_ind]
                V_add[y][q_ind]+=error_mat[x][y]*U[x][q_ind]
                 
        U=U+lr*(U_add)
        V=V+lr*(V_add)
#   predicted_matrix=np.dot(U,np.transpose(V))
#   return predicted_matrix
    return U,V
  

In [31]:
U_k,V_k=GradientDescent(ratings_matrix_train,346,1.5e-6,10000)

KeyboardInterrupt: 

In [23]:
result_gd_batch=[]
predicted_matrix=np.dot(U_k,np.transpose(V_k))
for i in test_nos:
  

  movie_no=inverse_movie_map[ratings_df['movieId'][i]]
  user_no=ratings_df['userId'][i]-1
  result_gd_batch.append(predicted_matrix[user_no][movie_no])
  

In [24]:
rmse_error(correct_predictions,result_gd_batch)

0.9633212134618514

In [25]:
np.save(path+'U_k',U_k)
np.save(path+'V_k',V_k)

In [106]:
path

'data/ml-latest-small/'

In [24]:
def sgd(R,k,lr,epochs):
    number_of_items=len(R[0])
    number_of_users=len(R[:,0])
    U=np.random.rand(number_of_users,k)
    U_temp=np.random.rand(number_of_users,k)
    V=np.random.rand(number_of_items,k)
    V_temp=np.random.rand(number_of_items,k)
    error_mat=np.zeros(shape=(number_of_users,number_of_items))
    num_iters=0
    while(num_iters<epochs):
        num_iters+=1
        for i in range(number_of_users):
            for j in range(number_of_items):
                error_mat[i][j]=R[i][j]-np.dot(U,np.transpose(V))[i][j]
                for q in range(k):
                    U_temp[i][q]=U[i][q]+lr*error_mat[i][j]*V[j][q]
                for q in range(k):
                    V_temp[i][q]=V[i][q]+lr*error_mat[i][j]*U[i][q]
                for q in range(k):
                    U[i][q]=U_temp[i][q]
                    V[i][q]=V_temp[i][q]
        print("Iteration Number: "+ str(num_iters)+" Frob Error: "+ str(LA.norm(error_mat,'fro')))
                
        

#   predicted_matrix=np.dot(U,np.transpose(V))
#   return predicted_matrix
    return U,V
    

In [25]:
U_k1,V_k1=sgd(ratings_matrix_filled,k,1.5e-5,10000)

KeyboardInterrupt: 